In [19]:
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras

In [20]:
# 定义RNN参数
HIDDEN_SIZE = 30                            # LSTM中隐藏节点的个数。
NUM_LAYERS = 2                              # Deep_LSTM的层数。
TIMESTEPS = 10                              # 循环神经网络的训练序列长度。
TRAINING_STEPS = 10000                      # 训练轮数。
BATCH_SIZE = 32                             # batch大小。
TRAINING_EXAMPLES = 10000                   # 训练数据个数。
TESTING_EXAMPLES = 1000                     # 测试数据个数。
SAMPLE_GAP = 0.01                           # 采样间隔。

In [21]:
# 正弦函数采样
def generate_data(seq):
    X = []
    y = []
    # 序列的第i项和后面的TIMESTEPS-1项合在一起作为输入；第i + TIMESTEPS项作为输
    # 出。即用sin函数前面的TIMESTEPS个点的信息，预测第i + TIMESTEPS个点的函数值。
    for i in range(len(seq) - TIMESTEPS):
        X.append([seq[i: i + TIMESTEPS]])
        y.append([seq[i + TIMESTEPS]])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [22]:
test_start = (TRAINING_EXAMPLES + TIMESTEPS) * SAMPLE_GAP
test_end = test_start + (TESTING_EXAMPLES + TIMESTEPS) * SAMPLE_GAP
train_X, train_y = generate_data(np.sin(np.linspace(
    0, test_start, TRAINING_EXAMPLES + TIMESTEPS, dtype=np.float32)))
test_X, test_y = generate_data(np.sin(np.linspace(
    test_start, test_end, TESTING_EXAMPLES + TIMESTEPS, dtype=np.float32)))

In [23]:
inputs=keras.Input(shape=(1,TIMESTEPS))
cell = tf.nn.rnn_cell.MultiRNNCell([
    tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)
    for _ in range(NUM_LAYERS)])
output= layers.RNN(cell)(inputs)
predictions=layers.Dense(1,activation=None)(output)
model=keras.Model(inputs=inputs, outputs=predictions,name="DeepRNN_model")


ValueError: Variable cell_0/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

In [24]:
model.summary()


Model: "DeepRNN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 10)]           0         
_________________________________________________________________
rnn (RNN)                    (None, 30)                12240     
_________________________________________________________________
dense (Dense)                (None, 1)                 31        
Total params: 12,271
Trainable params: 12,271
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adagrad(learning_rate=0.1),
    metrics=["accuracy"]
)


In [26]:
model.fit(train_X, train_y,batch_size=BATCH_SIZE,epochs=10)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "C:\sofeware_install\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 4025, in <genexpr>
    for ta, input_ in zip(input_ta, flatted_inputs))  File "C:\sofeware_install\Anaconda3\lib\site-packages\tensorflow_core\python\util\tf_should_use.py", line 198, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs))
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "C:\sofeware_install\Anaconda3\lib\site-packages\tensorflow_core\python\keras\backend.py", line 4025, in <genexpr>
    for ta, input_ in zip(input_ta, flatted_inputs

In [27]:
model.evaluate(test_X, test_y)


1000/1000 [==============================] - 0s 143us/sample - loss: 1.5931e-06 - acc: 0.0010


[1.5931376739217739e-06, 0.001]